In [1]:
import numpy as np
import math
from scipy.optimize import curve_fit

---------------------------------------GEM 1 : ENCODER AZIMUTH CORRECTION-------------------------------------------

In [2]:
def main():
    
    #Coleta dados de input
    jlist, jfilenum, filelist, jset, jjset, numframes = user_inputs()
    _, _, _, _, Dir4 = directories()
    tag, file_list, file_name, input_data, n_lines = input_files()
    n_files = len(file_list)
    jframe, time, azimut = input_datarr()
    
    # Para cada arquivo da filelist, faz as correções necessárias nos dados de azimut e
    # escreve os arquivos de output
    for file_name_index in range(n_files): # for each file in list file
        dpfazi,slope,dslope,j = azi_correction(frames,azimut,time)
        # Calcula e escreve nos arquivos de output as estatísticas relacionadas a rotação
        write_stat_rot(dpfazi,slope,dslope,j,file_name,Dir4)
        

In [3]:
def user_inputs():
    
    " Coleta inputs do usuário."
    
    jlist = int(input("Single file (1) or Multiple files (2)?: \n"))
    print("\n")
    
    if jlist == 1:
        datetag = int(input("Enter the date of the year [yy_MM_dd] : \n"))
        jfilenum = int(input("Enter the UT time of the day [hhmmssml] : \n"))
        jj = 1
    else:
        filelist = input("ENTER the filelist name: \n")
        print("\n")
        jj = 0

    jset = int(input(" SET Vmax? [1] => yes / [2] => no \n"))
    print("\n")

    if jset == 1:
        Vmax = float(input(" Write Vmax: \n"))
    else:
        Vmax = 9.8993 # 1999 observations

    jjset = int(input("SET slp rate? [1] => yes / [2] => no \n"))
    print("\n")
    if jjset == 1:
        slp = float(input("Write slp rate: \n"))
    else:
        slp = 17.0357 # iterated 1999 Vmax +/- 0.3918

    numframes = int(input("ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] \n"))
    
    return jlist, jfilenum, filelist, jset, jjset, numframes

In [4]:
def directories():
    
    " Diretórios."
    
    Dir = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/txt/" 
    Dir1 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/dat/"
    Dir2 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/"
    Dir3 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/azimuth/"
    Dir4 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/dpf/"
    
    return Dir, Dir1, Dir2, Dir3, Dir4

In [5]:
def input_files():
    
    " A partir das informações dadas pelo usuário, coleta os arquivos de input."

    _,_,filelist,_,_,_ = user_inputs()
    Dir, Dir1, Dir2, Dir3, Dir4 = directories()
    with open(Dir + filelist) as arq: 
        filelist = arq.readlines()
        n_files = len(filelist)
        file_list, datetag, filenum, tag = [], [], [], []  
        for file_name_index in range(n_files):
            s_datetag = str(filelist[file_name_index])
            datetag.append(s_datetag[:8])
            filenum.append(s_datetag[8:16])
            tag.append(s_datetag)
            file_list.append(filelist[file_name_index])
            #----------------------------
            file_name = filelist[file_name_index]
            file_name = file_name.replace("\t", "")
            file_name = file_name.replace("\n", "")
            with open(Dir1 + file_name, "r+") as f: 
                input_data = f.readlines()
                n_lines = len(input_data)
        
    return tag, file_list, file_name, input_data, n_lines

In [6]:
def input_datarr():
    
    " Coleta dados de input de cada um dos arquivos de uma determinada filelist."
 
    jframe, time, azimut = [],[],[]
    _,_,_,input_data,n_lines = input_file()
    for line in input_data:
        input_line = line.replace(" ", "")
        input_line = line.replace("\n", "")
        input_line = input_line.split(" ")
        input_line = list(filter(None, input_line))
        jjframe = float(input_line[0])
        jtime = float(input_line[1])
        jazimut = float(input_line[4])
                
        jframe.append(jjframe)
        time.append(jtime)
        azimut.append(jazimut)
    
    return jframe, time, azimut

In [7]:
def azi_correction(frames,azimut,time):
   
    " Para cada frame de um arquivo, verifica e corrige (com auxílio de outras funções) valores" 
    " de azimute incorretos."  
    
    timemean, aazmean, j, k, n_rotations = 0, 0, 0, 0, 0
    aazimut, ttime, jccor, sig = [], [], [], []
    first, dangling = True, False
    i_frames = len(jframe)
    for i in range(i_frames): #for each line in file
        if first: # Get a reference in time and direction.
            compaz = azimut[i]   
            firstime = time[i]
            first = False
            
        #this is the expected behavior (azi-compaz<176), except for the very 
        #last azimut which may differ from the previous one
        #by < 176 and yet show reversal in sense of 
        #direction. The usual is a continuous decrease in 
        #azimut. An unexpected sign reversal can also happen. =>
        
        if (abs(azimut[i]-compaz)) < 176:
            l,ic,dangling,ttime,timemean,aazmean,sig,aazimut = azi_expected(i,105,aazimut,timemean,aazmean,\
            dangling,ttime,jccor,sig)
        else:
            if l >= (numframes-1):
                j,slope,dslope,lastazi,midazi,firstazi,dpfazi = azi_signal_reverse(numframes,dangling,aazimut,\
                timemean,aazmean,ttime,azimut,l) 
                
                # write in rot file (descrições dos arq. a escrever aq)
                dat_list_rot = [j,slope,dslope,lastazi,midazi,firstazi,dpfazi[j]]
                rot_type = "rot.dat"
                mode_rot = True
                write_output(dat_list_rot,file_name,Dir2,rot_type,mode_rot)
                
            else:
                slope,dslope,aazi,l2 = azi_incomplete_rotations(timemean,aazmean,ttime,aazimut,azimut,l)
            
            aazimut,fpd = fpd(slope,l,l2,aazimut,ttime) 
            
            # write in azimut file
            dat_list_aazi = aazimut
            if (n_rotations) == 0: mode_aazi = True
            else: mode_aazi = False
            aazi_type = "azi.dat"
            write_output(dat_list_aazi,file_name,Dir3,aazi_type,mode_aazi)
            
            if i < i_frames:
                l,ic,timemean,aazmean,dangling = 0,0,0,False # reset próxima rotação
                n_rotations =+ 1
                
        compaz = azimut[i] # definindo compaz para o próximo valor de azimut (prox. linha)
    
    # last incomplete rotation
    aazimut = last_incpl_rotation(l,aazimut)
    # write last incomplete rotation
    dat_list_azi = aazimut
    if (n_rotations) == 0: mode_azi = True
    else: mode_azi = False
    azi_type = "azi.dat"
    write_output(dat_list_azi,file_name,Dir3,azi_type,mode_azi)
    
    return dpfazi,slope,dslope,j

In [8]:
def azi_expected(i,numframes,aazimut,timemean,aazmean,dangling,ttime,jccor,sig):
    
    " Verifica se há uma reversão no sinal no último azimut (da rotação),"
    " ou se há uma inversão inesperada "
    
    ic = None 
    ind_tt = len(ttime)
    if i == 0:          # valor de l => contabilização dos azis que decrescem dentro do esperado com o tempo 
        l = 0           # dentro de uma rotação do radiotelescópio.
    else:
        l += 1 
    
    if (l > 0) and (time[i] - ttime[l-1] < 0):
        ttime.append(time[i] + 86400)
    else:
        ttime.append(time[i])
    aazimut[l] = azimut[i]
    timemean = timemean + ttime[l]
    aazmean = aazmean + azimut[i]
    sig[l] = 1
    if azimut[i]-compaz > 0: #
        if l >= (numframes-1):
            dangling = True #sign reversal in last azimut detected 
        else: 
            if ic == None: ic = 0 #unexpected sign
            else: ic += 1
            jccor[ic] = l
            
    return l,ic,dangling,ttime,timemean,aazmean,sig,aazimut

In [9]:
def azi_signal_reverse(numframes,dangling,aazimut,timemean,aazmean,ttime,azimut,l):
    
    " Caso o valor atual de azi tenha decrescido fora do esperado (azi-compaz > 176),"
    " mas o número atual de 'l' contabilizados corresponde a uma rotação completa,"
    " então será feita uma correção nos valores de azi em que ocorreram reversões inesperadas,"
    " a partir do valor calculado de slope (vel./s) na função dpf."
    
    j105, j106, lastazi, midazi, firstazi, j = 0, 0, 0, 0, 0, None
    speed, dspeed, dpfazi = [],[],[]
                            #In case the sign reversal had already occured
    if l == (numframes-1):  #in the previous frame but the magnitude fell 
        j105 += 1           #short of 176, we delete it from the mean (mm=1)
    if l == numframes:      #double-dangling forces l to increase by 1. Even
        j106 += 1           #when lastazi is really only the l-1 element, mid
    if j == None: j = 0      #azi serves just as well as firstazi to set dpfazi
    else: j+=1

    if dangling:# Se ocorreu reversão no último frame
        mm = 1
        slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm)
        lastazi = aazimut[l-2] 
        midazi = aazimut[l-1]
        firstazi = aazimut[l]
        l2 = 1
    else: # Se não
        lastazi = aazimut[l]
        mm = 0
        slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm)
        midazi = aazimut[l]
        firstazi = aazimut[l+1]
        l2 = 2

        speed[j] = slope
        dspeed[j] = dslope 
        #If the Vmax used is correct, then midazi will be
        #spaced evenly from lastazi to firstazi according
        #to the prevailing slope. So we check and correct
        dpfazi[j] = ((360.0 + lastazi - firstazi)/2) # Calculando vel. graus/frame

        # ic value = times unexpected signal reverse occurs (first time => ic = 0) 
        if ic >= 0: 
        # Substituindo valores de azimute onde aconteceram reversões inesperadas, pelo valor corrigido 
        # de (vel./s * tempo_entre_frames = azi_corrigido)
            for i in range(ic):
                aazimut[(jccor[i])] = aazimut[(jccor[i] - 1)] + (slope*0.56002)
                if aazimut[(jccor[i])] < 0: aazimut[(jccor[i])] += 360.0
                        
    return j,slope,dslope,lastazi,midazi,firstazi,dpfazi[j] # write in rots file

In [10]:
def azi_incomplete_rotations(timemean,aazmean,ttime,aazimut,azimut,l):
    
    " Caso 'l' corresponda a uma rotação incompleta, o valor de slope também será calculado. "
    
    print("Incomplete rotation of l frames =>",l)
    
    if ic >= 0:
        mm = 1
        slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm)
        l2 = 1
    else:
        mm = 0
        slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,azimut,l,mm)
        l2 = 2
    
    return slope,dslope,aazi,l2

In [11]:
def fpd(slope,l,l2,aazimut,ttime):
    
    " Calcula fpd (degree/frame), a partir do valor de slope, e realiza uma correção nos "
    " valores de azi que não forem compatíveis, numa margem, com o azimute calculado a partir de (vel*tempo)."
    
    fpd = slope*0.56002 
    d = 0
    while d <= (l+l2):
        if d > 0:
            aa_zi = abs(aazimut[d] - aazimut[d-1]) 
            tti_me = (ttime[d] - ttime[d-1])
            
            if (aa_zi < (abs(fpd/2))) or (aa_zi > (3*(abs(fpd/2)))) and (tti_me < (3*0.56002/2)):
                
                aazimut[d] = aazimut[d-1] + fpd 
        d += 1
        
    return aazimut,fpd

In [12]:
def last_incpl_rotation(l,aazimut):
    
    " Correção dos valores da última rotação."
    
    while i <= l: #last incomplete rotation 
        if i > 0:
            
            if ((aazimut[i] - aazimut[i-1]) < (fpd/2)) or ((aazimut[i] - aazimut[i-1]) > (3*fpd/2))\
            and ((ttime[i] - time[i-1]) < (3*0,56002/2)):
                aazimut[i] = aazimut[i-1] + fpd
        i += 1
        
    return aazimut

In [13]:
def write_output(listt,file_name,dirr,type_file,mode):
    
    " Escreve os dados processados em seus respectivos arquivos"
    
    data = str(listt[0])
    for item in listt:
        item = f"{item:.2f}"
        data += f"\t{item}\t"
    new_name = file_name.replace(".dat", type_file)

    if mode:
        with open(dirr + new_name,"w") as arq_output:
            arq_output.write(data)
            arq_output.write("/n")
    else:
        with open(dirr + new_name,"a") as arq_output:
            arq_output.write(data)
            arq_output.write("/n")

In [14]:
def write_stat_rot(dpfazi,speed,dspeed,j,file_name,Dir4):
    
    slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,\
    mean_per,per_sigman,sigma_per,sigint_per,Vslp = stats_parameters(dpfazi,speed,dspeed,j)
    
    mode_rot_dpf = True
    listt_dpf = [fpd,sig_dpf,dpfmean,sigdpf,(fpd-dpfmean),Vslp]
    type_dpf = "dpf_rots.dat"
    dirr_dpf = Dir4
    write_output(listt_dpf,file_name,Dir4,type_dpf,mode_rot_dpf)
    

In [15]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::				
def stats_parameters(dpfazi,speed,dspeed,j):
    """
    Calculates rotation statistics
    """
    wt, slopemean, dpfmean, sigma, sigdpf = 0,0,0,0,0
    for i in range(j):
        w = 1.0/dspeed[i]**2
        slopemean += speed[i]*w
        wt += w 
        dpfmean += dpfazi[i]
    print("w",w)
    slopemean = slopemean/wt #mean dish rotation speed in deg/sec
    dpfmean = dpfmean/j ##Average dpf speed in deg/frame
    for i in range(j):
        w = 1.0/dspeed[i]**2
        sigma = sigma + w*(speed[i] - slopemean)**2    
        sigdpf = sigdpf + (dpfazi[i] - dpfmean)**2

    sigma = math.sqrt(sigma/wt*j/(j-1)) # sigma dispersion (slopemean) deg/sec
    sigman = sigma/math.sqrt(j*1) # external error (slopemean) deg/sec
    sigint = math.sqrt(1/wt) # internal error (slopemean) deg/sec            
    Z = sigman/sigint # z statistic
    dZ = 1/math.sqrt(2*(j-1))         
    sigdpf = math.sqrt(sigdpf/(j-1))/math.sqrt(j*1)

    fpd = slopemean*0.56002 # frames/degree
    sig_dpf = sigma * 0.56002 # sigma dispersion
    sigman_dpf = sigman * 0.56002 # external error
    sigint_dpf = sigint * 0.56002 # internal error

    mean_per = 360/slopemean # Average rotation period in seconds
    per_sigman = 360*sigman/slopemean**2 # external error
    sigma_per = 360*sigma/slopemean**2 # sigma dispersion
    sigint_per = 360*sigint/slopemean**2 # internal error

    Vslp = Vmax + (fpd - dpfmean)/slp
    return(slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,mean_per,per_sigman,sigma_per,sigint_per,Vslp)

In [16]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def dpf(tmean,zmean,ttim,aazi,azi,n,m):
    """
    Using the output of the least_sq function and the average values ​​
    of time and azimuth, calculate the speed in degrees/frame
    """
    n_azi = len(azi)
    for fr_azi in range(n_azi):
        if m == 0: #se não há inversão de sinal
            tmean = tmean/n ##
            zmean = zmean/n ##
            jh = (n-1)+2-m
            aazi[jh] = azi[fr_azi]
        else: #se há
            tmean = (tmean - ttim[n])/(n-1) ##
            zmean = (zmean - aazi[n])/(n-1) ##
            jh = (n-1)+2-m
            aazi[jh] = azi[fr_azi]
        i = 0 ##
        while i < n-m: ##
            ttim[i] = (ttim[i] - tmean)
            aazi[i] = (aazi[i] - zmean)
            i += 1
        ttim = np.array(ttim, dtype="f8")
        aazi = np.array(aazi, dtype="f8")
        if (n-m) > 2: ##mod##  ##########################################################################################
            mnt = 0
            zer, slope, dzer, dslope  = least_sq(function, ttim, aazi)
        else:
            slope, dslope = 0, 0

        i = 1
        while i < (n+1)-m:
            aazi[i] += zmean
        aux = azi[fr_azi]
        jk = (n)-m
        if (aazi[jh]<=azi[fr_azi]) or (aazi[jh]-azi[fr_azi])<(slop*0.56002/8*7):
            dstep = (360+aazi[jk]- azi[fr_azi])/3
            for i in [0,1,2]:
                jr = (n)+i-m
                aazi[jr] = aazi[jk] - i*dstep
                if aazi[jr]<0:
                    aazi[jr] = aazi[jr]
        else:
            jr = (n+1)+1-m
            ji = (n+1)+2-m
            aazi[jr] = aazi[jk] - (360 + aazi[jk] - aazi[ji])/2
            if aazi[jr]<0:
                aazi[jr] = aazi[jr]
            jg = (n+1)+3-m
            aazi[jg] = azi[fr_azi]
            n += 1

    return slope,dslope,aazi

In [17]:
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::	
def least_sq(function,x,y):
    """
    Calculate a linear fit to the azimuth data as a function of time,
    obtaining the speed in degrees/sec and their respective uncertainties
    """
    param, m_cov = curve_fit(function, x, y)
    print("a,b", param)
    print(param[0])
    #a = param[0], b = param[1]
    perr = np.sqrt(np.diag(m_cov)) #siga, sigb
    print("siga, sigb", perr)
    #siga, sigb = perr[0], perr[1]

    #return a,b,siga,sigb
    return param[0],param[1],perr[0],perr[1]
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [18]:
def function(x,a,b):
    return a + b*x

#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [19]:
main()

Single file (1) or Multiple files (2)?: 
2


ENTER the filelist name: 
fileli001.txt


 SET Vmax? [1] => yes / [2] => no 
2


SET slp rate? [1] => yes / [2] => no 
2


ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] 
105


UnboundLocalError: cannot access local variable 'datetag' where it is not associated with a value